In [1]:
import requests
import re

class MyCrawler:
    def __init__(self, filename):
        self.filename = filename
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4128.3 Safari/537.36'
        }

    def download(self, url: str):
        r = requests.get(url, headers=self.headers)
        return r.text

    def extract(self, content, pattern):
        res = re.findall(pattern, content)
        return res

    def save(self, info):
        with open(self.filename, 'a', encoding='utf-8') as f:
            for item in info:
                f.write(''.join(item) + '\n') 

    def crawl(self, url: str, pattern: str, headers=None):
        if headers:
            self.headers.update(headers)
        content = self.download(url)
        info = self.extract(content, pattern)
        self.save(info)

In [2]:
dou_crawler = MyCrawler('douban.txt')

url = 'https://book.douban.com/tag/%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C'

headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4128.3 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://book.douban.com/',
    'Accept-Language': 'zh-CN,zh;q=0.9',
}

content = dou_crawler.download(url)
assert('Neural Networks and Deep Learning' in content)

In [3]:
re.findall (
    'src="(.*?).jpg"[\S\s]*?<a href="(.*?)"\stitle="(.*?)"[\S\s]*?<div\sclass="pub">\s*(.*?)\s*<\/div>[\S\s]*?<div\sclass="star\sclearfix">\s*([\S\s]*?)\s*<\/div>',content)

[('https://img1.doubanio.com/view/subject/s/public/s33631858',
  'https://book.douban.com/subject/35044046/',
  '神经网络与深度学习',
  '邱锡鹏 / 机械工业出版社 / 2020-4-10 / 149.00元',
  '<span class="allstar45"></span>\n        <span class="rating_nums">9.4</span>\n\n    <span class="pl">\n        (162人评价)\n    </span>'),
 ('https://img9.doubanio.com/view/subject/s/public/s29738046',
  'https://book.douban.com/subject/30192800/',
  'Python神经网络编程',
  '[英]塔里克·拉希德（Tariq Rashid） / 林赐 / 人民邮电出版社 / 2018-4 / 69.00元',
  '<span class="allstar45"></span>\n        <span class="rating_nums">9.2</span>\n\n    <span class="pl">\n        (459人评价)\n    </span>'),
 ('https://img1.doubanio.com/view/subject/s/public/s29839337',
  'https://book.douban.com/subject/30293801/',
  'Python深度学习',
  '[美] 弗朗索瓦•肖莱 / 张亮 / 人民邮电出版社 / 2018-8 / 119.00元',
  '<span class="allstar50"></span>\n        <span class="rating_nums">9.6</span>\n\n    <span class="pl">\n        (595人评价)\n    </span>'),
 ('https://img9.doubanio.com/view/subject/s/pu

In [6]:
class MyDoubanCrawler(MyCrawler):
    def extract(self, content, pattern_main, pattern_star):
        res = re.findall(pattern_main, content)
        for index in range(len(res)):
            if 'allstar' in res[index][4]:
                items = re.findall(pattern_star, res[index][4])
            else:
                items = [['0', '0', '0']]
            res[index] = list(res[index])
            del res[index][4]
            res[index].extend(items[0])
        return res
            
    def crawl(self, url, pattern_main, pattern_star, headers=None):
        if headers:
            self.headers.update(headers)
        content = self.download(url)
        info = self.extract(content, pattern_main, pattern_star)
        self.save(info)

In [7]:
dou_crawler = MyDoubanCrawler('douban.txt')

dou_crawler.crawl(
    'https://book.douban.com/tag/%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C',
    'src="(.*?).jpg"[\S\s]*?<a href="(.*?)"\stitle="(.*?)"[\S\s]*?<div\sclass="pub">\s*(.*?)\s*<\/div>[\S\s]*?<div\sclass="star\sclearfix">\s*([\S\s]*?)\s*<\/div>',
    '<span class="allstar(\d+)".*?[\S\s]*?rating_nums">(.*?)<\/span>[\S\s]*?\((\d+)',
)